## __repr__，__str__的区别和用法

In [36]:
class Pair: 
    def __init__(self, x, y): 
        self.x = x 
        self.y = y 
    def __repr__(self): 
        return 'Pair({0.x!r}, {0.y!r})'.format(self) 
    def __str__(self): 
        return '({0.x!s}, {0.y!s})'.format(self)

p = Pair(1,2)
print(repr(p))
print(p)

Pair(1, 2)
(1, 2)


## __format__ 的用法

In [37]:
_formats = { 
    'ymd' : '{0.year}-{0.month}-{0.day}', 
    'mdy' : '{0.month}/{0.day}/{0.year}', 
    'dmy' : '{0.day}/{0.month}/{0.year}' 
} 
class Date: 
    def __init__(self, year, month, day): 
        self.year = year 
        self.month = month 
        self.day = day 
    def __format__(self, code): 
        if code == '': 
            code = 'ymd' 
        fmt = _formats[code] 
        return fmt.format(self)
    
d = Date(2017,4,17)
print(format(d))
print(format(d, 'dmy'))
print('Today is {:dmy}'.format(d)) #传参数的格式 {:arg1 arg2}

2017-4-17
17/4/2017
Today is 17/4/2017


## @property 的用法

使用场景：你想给某个实例 attribute 增加除访问与修改之外的其他处理逻辑，比如类型检查 或合法性验证。

描述符：@property是一个语法，property本身是一个描述符(descriptor)，描述符就是一个有`__get__`,`__set__`这样函数的类，根据文档，描述符只有声明为类的属性的时候，它的行为才能得到保证。

In [33]:
class Person:
    def __init__(self, first_name):
            self.first_name = first_name

    # Getter function
    @property
    def first_name(self):
        return self._first_name
    
    # Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string') 
        self._first_name = value

    # Deleter function (optional)
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")

In [34]:
p = Person('Xiao')
p.first_name = 123

TypeError: Expected a string

In [249]:
class Parrot:
    def __init__(self):
        self._voltage = 100000
    def fget(self):
        """Get the current voltage."""
        return self._voltage
    def fset(self, value):
        """Set the current vlotage"""
        self._voltage = value
    voltage = property(fget,fset)

parrot = Parrot() # 把方法变成了属性，然后这些方法是用来管理这个属性的。
parrot2 = Parrot()
parrot.voltage = 99999
parrot2.voltage
parrot.voltage    # 类的实例是各自变化的。

Parrot.voltage.__set__(Parrot, 2) 
# 这里的过程，首先 Parrot.voltage 会调用 voltage.__get__，然后返回 property 对象本身，(见下方 property 的实现)
# 然后会调用 property 本身的 __set__ 函数，传入参数是 (Parrot, 2)，__set__ 里调用了 fset ，fset 就是 Parrot 的实际设置函数
# 相同的参数传入到 fset 里，然后就可以看到下面的结果 Parrot._voltage = 2
Parrot._voltage

Parrot.voltage = 1 # 注意！这里Parrot.voltage和上面一样得到的是 property 本身，然后对描述符自身赋值，是不会调用 __set__ 函数的
                   # 而是将数字 1 绑定到了这个 voltage 描述符上。
parrot.voltage     # 所有实例都会变，因为引用的都是类空间里的 voltage 变量，现在已经指向数字 1 了。
parrot2.voltage    # 同上。
type(Parrot.voltage) == type(parrot.voltage) == int


100000

99999

2

1

1

True

[property 的实现，Gist Link](https://gist.github.com/tvytlx/1abbe7b0ae4c8837dacc174cad65e0ad)

## super()
super().method() 或者 super(Cls, self).method() 在类里的时候是一样的，因为前者会被编译器填充内容，而在类外动态添加的时候，前者则需要显示指明那两个参数`Cls, self`。

In [243]:
class C(object):
    def __init__(self):
        pass
    def print_(self):
        print('This is Class: C')
class A(C):
    def test(self):
        # 参数可以省略
        super().print_()
        
a = A()
a.test()
def print_(self):
    # 这里的参数不能少。
    super(A, self).print_() # 给类添加一个方法，意味着给所有实例*绑定*一个新方法
A.print_ = print_
a.print_()

This is Class: C
This is Class: C


还有一个问题是，怎么在子类中扩展父类描述符。

```python
class Person:
    def __init__(self, name):
        self.name = name
    # Getter function
    @property
    def name(self):
        return self._name
    # Setter function
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value
    # Deleter function
    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")

class SubPerson(Person): 
    @property
    def name(self):
        print('Getting name')
        return super().name
    @name.setter
    def name(self, value):
        print('Setting name to', value)
        # 这里这么写是因为，只有通过父类的类变量才能拿到描述符，用父类的实例变量拿不到。
        super(SubPerson, SubPerson).name.__set__(self, value)
    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)
```